# Pinterest Data Analysis

In [182]:
import pandas as pd
import requests as rq
import json
from IPython.display import display, HTML, Javascript
import nvd3
nvd3.ipynb.initialize_javascript(use_remote=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Call the Clarabridge API for processed topic information:

In [183]:
payload = {'additionalFilters':["_id_source:pinterest"],
	 'additionalMetrics': [69, 71],
	 'attribute': 1,
	 'count': '20',
	 'direction': 'DESC',
	 'filters': [],
	 'modelId': 1295359,
	 'nodeName': 'Munchies Model',
	 'page': {'lookAheadLimit': 20},
	 'project': 48492,
	 'projectName': 'SOTW Lite Data',
	 'reportType': 'TOPICLEAF',
	 'sample': 'full',
	 "showOther":False,
	 'selectedNodes': {},
	 'sort': 'volume',
	 'topBottom': 'Top',
	 'widgetType': 'standard',
	 'metric3':'repins',
	 'metric4':'likes'}

headers = {'content-type': 'application/json'}
url = 'https://dexter.clarabridge.net/cbrestfulapi/v1/report'
authDexter = ("dheepan.dexter","Recalitrantkitten5")
apicall = rq.post(url,auth=authDexter,headers=headers,data=json.dumps(payload))
print apicall.status_code
#check for successful request

200


## Overall Top Pins

In [184]:
pinterestDf = pd.read_json(json.dumps(apicall.json()['data']))
#drop extraneous columns, name == category
pinterestDf = pinterestDf.drop(["fullPath", "sentiment","sent_count","id","volume"],1)

newcols = pinterestDf.columns.values
newcols[0:3] = ["Recipes","Likes","Repins"]
pinterestDf.cols = newcols
#trim bad categories

pinterestDf = pinterestDf[pinterestDf["name"] != "Nuts"]
pinterestDf2 = pinterestDf.set_index("name")
display(pinterestDf2.ix[0:10])


,Recipes,Likes,Repins,percentOfVolume
name,,,,
Dips,325,107.71,1319.13,14.10
Bacon,124,197.70,2348.77,5.38
Wings,105,182.97,2075.87,4.56
Chilli,101,105.15,1207.81,4.38
Tacos,95,225.33,2667.35,4.12
Pretzels,85,197.64,2317.81,3.69
Cookies,83,207.31,1686.72,3.60
Pizza,81,199.43,2304.96,3.51
Crackers,62,179.44,1741.90,2.69


* Dips of various kinds are the #1 category of recipe by volume
* America's bacon obession comes out clear, an average recipe with Bacon has 2350 repins and the second most recipes
* Meatballs are also very popular and have the most repins on average (less recipes)
* Desserts are decidely less popular and only Cupcakes make the top list

In [188]:
#build barchart
extra = "chart.reduceXTicks(false);"
chart = nvd3.multiBarChart(width=900, height=400, x_axis_format=None, 
xAxis_rotateLabel=45, margin_bottom=100)
x_axis = pinterestDf["name"].tolist()
chart.add_chart_extras(extra)
y_1 = pinterestDf["Recipes"].tolist()
y_2 = pinterestDf["Repins"].tolist()
chart.add_serie(name="Recipes", y=y_1, x=x_axis)
chart.add_serie(name="Repins", y=y_2, x=x_axis)

chart.buildcontent()
display(HTML(chart.htmlcontent))

#not bothering with making mutliple axis', this a draft, click Recipes or Repins to better look at data

## What are the top dips?

In [191]:
payload2 = {'additionalFilters':['_id_source:pinterest','_catRef:[model:"Munchies Model" path:"root" node:"Dips"]'],
	 'additionalMetrics': [69, 71],
	 'attribute': 1,
	 'count': '50',
	 'direction': 'DESC',
	 'filters': [],
	 'modelId': 1295359,
	 'nodeName': 'Munchies Model',
	 'page': {'lookAheadLimit': 50},
	 'project': 48492,
	 'projectName': 'SOTW Lite Data',
	 'reportType': 'associatedWords',
	 'sample': 'full',
	 "showOther":False,
	 'selectedNodes': {},
	 'sort': 'volume',
	 'topBottom': 'Top',
	 'widgetType': 'standard',
	 'metric3':'repins',
	 'metric4':'likes'}
apicall2 = rq.post(url,auth=authDexter,headers=headers,data=json.dumps(payload2))
print apicall2.status_code


200


In [192]:
dips = pd.read_json(json.dumps(apicall2.json()['data']))
#drop extraneous columns, name == category
dips = dips.drop(["sentiment","sent_count","id","volume"],1)

dips = dips.set_index('name')
dips.sort_values(by="metric4", ascending=False).ix[0:10]


,doc_count,metric3,metric4,percentOfVolume
name,,,,
DIP->DELICIOUS,5,922.00,12280.20,1.54
APPETIZER->PARTY,4,525.50,6093.75,1.23
BACON JALAPENO CORN DIP->CHEESY,3,363.67,3937.00,0.92
DIP->BLACK BEAN,3,295.00,3780.67,0.92
PARTY->NEXT,4,268.00,3556.75,1.23
CHICKEN->BUFFALO,7,213.57,2453.57,2.15
DIP->CHICKEN,8,198.00,2271.75,2.46
DIP->CHEESE,5,171.20,2132.40,1.54
DIP->DIP,6,154.50,2093.50,1.85


Cheese dips are definitely the main attraction when looking at pins, which is no surprise. In second place is black-bean dips, followed by Chicken dip.

## What are the main ingredients?

In [195]:
payload3 = {'additionalFilters':["_id_source:pinterest","_verbatimtype:ingredientslist"],
	 'additionalMetrics': [69, 71],
	 'attribute': 1,
	 'count': '200',
	 'direction': 'DESC',
	 'filters': [],
	 'modelId': 1295359,
	 'nodeName': 'Munchies Model',
	 'page': {'lookAheadLimit': 200},
	 'project': 48492,
	 'projectName': 'SOTW Lite Data',
	 'reportType': 'words',
	 'sample': 'full',
	 "showOther":False,
	 'selectedNodes': {},
	 'sort': 'volume',
	 'topBottom': 'Top',
	 'widgetType': 'standard',
	 'metric3':'repins',
	 'metric4':'likes'}
apicall3 = rq.post(url,auth=authDexter,headers=headers,data=json.dumps(payload3))
ingredients = pd.read_json(json.dumps(apicall3.json()['data']))
#drop cups, tsp etc
pattern = r'\d'
ingredients2 = ingredients[ingredients.name.str.match(pattern, as_indexer=True) == False].ix[10:]
newcols = ingredients2[["name","percentOfVolume","metric4"]].columns.values
newcols = ["z","x","y"]
scatterdata = ingredients2[["name","percentOfVolume","metric4"]]
scatterdata.columns = newcols
scatterdata.to_json("scatterdata.json",orient = "records")
ingredients2.to_excel("ingredientslist.xlsx")
ingredients2

,doc_count,id,metric3,metric4,name,percentOfVolume,sent_count,sentiment,volume
11,247,-9,204.12,2464.88,MEAT,32.93,320,0.10,247
14,211,-16,147.82,1741.52,GARLIC,28.13,216,0.13,211
15,191,-19,106.06,1217.13,POWDER,25.47,201,0.10,191
16,191,-15,161.80,1868.52,SALT,25.47,233,0.11,191
17,185,-21,230.22,2566.14,BUTTER,24.67,194,0.02,185
19,177,-24,226.50,2777.35,BAKED GOODS,23.60,175,0.13,177
20,177,-22,140.59,1717.91,ONION,23.60,190,0.23,177
21,173,-17,173.94,1856.31,VINEGAR,23.07,214,0.07,173
22,171,-20,178.43,2137.27,PEPPER,22.80,197,0.12,171
24,164,-28,230.14,2812.11,BREAD,21.87,161,0.09,164


In [196]:
display(HTML("ingredientsScatterPlot.html")) #see full graphic

* Cheese consists of the 40% of the top recipes (cream cheese or plain cheese)
* Kosher recipes were also surprisingly popular
* Garlic is even more popular than salt
* Cumin & Cayenne were popular spices which points to the preponderance of Tacos in the pins